In [1]:
from multicall import Call

from mainnet_launch.data_fetching.get_state_by_block import get_raw_state_by_blocks, build_blocks_to_use
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import streamlit as st
from multicall import Call
from web3 import Web3


from mainnet_launch.constants import (
    AutopoolConstants,
    WETH,
    ROOT_PRICE_ORACLE,
    ChainData,
    ALL_AUTOPOOLS,
    BASE_ETH,
    eth_client,
    ETH_CHAIN,
)

from mainnet_launch.data_fetching.get_state_by_block import (
    get_raw_state_by_blocks,
    safe_normalize_with_bool_success,
    get_state_by_one_block,
    identity_with_bool_success,
)

from mainnet_launch.data_fetching.get_events import fetch_events

from mainnet_launch.abis import (
    AUTOPOOL_ETH_STRATEGY_ABI,
    ERC_20_ABI,
    BALANCER_AURA_DESTINATION_VAULT_ABI,
    ROOT_PRICE_ORACLE_ABI,
)
from mainnet_launch.destinations import get_destination_details
from mainnet_launch.data_fetching.add_info_to_dataframes import (
    add_timestamp_to_df_with_block_column,
    add_transaction_gas_info_to_df_with_tx_hash,
)
from mainnet_launch.database.new_databases import (
    write_dataframe_to_table,
    run_read_only_query,
    get_earliest_block_from_table_with_autopool,
)
from mainnet_launch.database.should_update_database import should_update_table


blocks = build_blocks_to_use(BASE_ETH.chain)
len(blocks)

2025-01-09 14:15:22.430 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-01-09 14:15:22.513 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-09 14:15:22.674 
  command:

    streamlit run /home/parker/Documents/Tokemak/v2-rebalance-dashboard/.venv/lib/python3.10/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-01-09 14:15:22.674 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-09 14:15:22.675 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-09 14:15:22.676 No runtime found, using MemoryCacheStorageManager
2025-01-09 14:15:23.176 Thread 'Thread-4': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-09 14:15:23.176 Thread 'Thread-4': missing Script

268

In [2]:
root_price_oracle_contract = BASE_ETH.chain.client.eth.contract(
    ROOT_PRICE_ORACLE(BASE_ETH.chain), abi=ROOT_PRICE_ORACLE_ABI
)
tokens: list[str] = fetch_events(root_price_oracle_contract.events.TokenRegistered)["token"].values
symbol_calls = [Call(t, ["symbol()(string)"], [(t, identity_with_bool_success)]) for t in tokens]

token_address_to_symbol = get_state_by_one_block(symbol_calls, max(blocks), BASE_ETH.chain)
token_address_to_symbol

2025-01-09 14:15:23.493 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-09 14:15:23.494 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-09 14:15:23.494 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-09 14:15:23.496 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-09 14:15:23.496 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


{'0x04C0599Ae5A44757c0af6F9eC3b93da8976c150A': 'weETH',
 '0xc1CBa3fCea344f92D9239c08C0568f6F2F0ee452': 'wstETH',
 '0xEDfa23602D0EC14714057867A78d01e94176BEA0': 'wrsETH',
 '0x2Ae3F1Ec7F1F5012CFEab0185bfc7aa3cf0DEc22': 'cbETH',
 '0xB6fe221Fe9EeF5aBa221c348bA20A1Bf5e73624c': 'rETH',
 '0x940181a94A35A4569E4529A3CDfB74e38FD98631': 'AERO',
 '0x4554485F494e5f55534400000000000000000000': None,
 '0x1509706a6c66CA549ff0cB464de88231DDBe213B': 'AURA',
 '0x4158734D47Fc9692176B5085E0F52ee0Da5d47F1': 'BAL',
 '0x223C0d94dbc8c0E5df1f6B2C75F06c0229c91950': 'TOKE',
 '0x4200000000000000000000000000000000000006': 'WETH',
 '0xEeeeeEeeeEeEeeEeEeEeeEEEeeeeEeeeeeeeEEeE': None,
 '0xC771c1a5905420DAEc317b154EB13e4198BA97D0': 'rETH-WETH-BPT',
 '0xaB99a3e856dEb448eD99713dfce62F937E2d4D74': 'weETH/wETH',
 '0x833589fCD6eDb6E08f4c7C32D4f71b54bdA02913': 'USDC',
 '0x54D86E177cdC664B5F9B17eB5fd6A76Fa529e466': 'ECLP-cbETH-wstETH'}

In [3]:
token_address_to_symbol = {
    "0x04C0599Ae5A44757c0af6F9eC3b93da8976c150A": "weETH",
    "0xc1CBa3fCea344f92D9239c08C0568f6F2F0ee452": "wstETH",
    "0xEDfa23602D0EC14714057867A78d01e94176BEA0": "wrsETH",
    "0x2Ae3F1Ec7F1F5012CFEab0185bfc7aa3cf0DEc22": "cbETH",
    "0xB6fe221Fe9EeF5aBa221c348bA20A1Bf5e73624c": "rETH",
    "0x4200000000000000000000000000000000000006": "WETH",
}


def getPriceInEth_call(name: str, token_address: str, chain: ChainData) -> Call:
    return Call(
        ROOT_PRICE_ORACLE(chain),
        ["getPriceInEth(address)(uint256)", token_address],
        [(name, safe_normalize_with_bool_success)],
    )


price_calls = [
    getPriceInEth_call(token_address_to_symbol[t], t, BASE_ETH.chain) for t in tokens if t in token_address_to_symbol
]

price_df = get_raw_state_by_blocks(price_calls, blocks, BASE_ETH.chain)
price_df

,weETH,wstETH,wrsETH,cbETH,rETH,WETH
timestamp,,,,,,
2024-11-02 23:52:33+00:00,1.051737,1.183100,1.025140,1.081698,1.118538,1.0
2024-11-03 05:52:33+00:00,1.051737,1.183100,1.025140,1.082375,1.118538,1.0
2024-11-03 11:52:33+00:00,1.051737,1.183100,1.025140,1.082375,1.118538,1.0
2024-11-03 17:52:33+00:00,1.051737,1.183296,1.026126,1.082375,1.117700,1.0
2024-11-03 23:52:33+00:00,1.051609,1.183296,1.026126,1.082375,1.117700,1.0
...,...,...,...,...,...,...
2025-01-07 17:52:33+00:00,1.056377,1.189403,1.032747,1.084183,1.120700,1.0
2025-01-07 23:52:33+00:00,1.056729,1.189403,1.032747,1.084183,1.120500,1.0
2025-01-08 05:52:33+00:00,1.056729,1.189403,1.032747,1.084183,1.120500,1.0


In [4]:
ROOT_PRICE_ORACLE(BASE_ETH.chain)

'0xBCf67d1d643C53E9C2f84aCBd830A5EDC2661795'

In [5]:
import plotly.express as px

px.line(price_df["rETH"])

In [6]:
px.line(price_df)

In [7]:
100 * (1.122 - 1.119)  # down 30 bips since jan 1?

0.30000000000001137

In [8]:
px.line(price_df)

# imo this looks bad, why is it doing that?

In [9]:
px.line(price_df["wstETH"])

In [10]:
rETH_CALCULATOR = "0x9801098EE481ed6806C61A4dE259FBdDD5bb84a8"

abi = """[{"inputs":[{"internalType":"contract ISystemRegistry","name":"_systemRegistry","type":"address"}],"stateMutability":"nonpayable","type":"constructor"},{"inputs":[],"name":"AccessDenied","type":"error"},{"inputs":[],"name":"IncorrectTimestamps","type":"error"},{"inputs":[{"internalType":"string","name":"paramName","type":"string"}],"name":"InvalidParam","type":"error"},{"inputs":[{"internalType":"bytes32","name":"role","type":"bytes32"},{"internalType":"address","name":"user","type":"address"}],"name":"MissingRole","type":"error"},{"inputs":[],"name":"NoSnapshotTaken","type":"error"},{"inputs":[],"name":"UndefinedAddress","type":"error"},{"inputs":[{"internalType":"string","name":"paramName","type":"string"}],"name":"ZeroAddress","type":"error"},{"inputs":[],"name":"ZeroDivisor","type":"error"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"uint256","name":"priorEthPerToken","type":"uint256"},{"indexed":false,"internalType":"uint256","name":"priorTimestamp","type":"uint256"},{"indexed":false,"internalType":"uint256","name":"currentEthPerToken","type":"uint256"},{"indexed":false,"internalType":"uint256","name":"currentTimestamp","type":"uint256"},{"indexed":false,"internalType":"uint256","name":"priorBaseApr","type":"uint256"},{"indexed":false,"internalType":"uint256","name":"currentBaseApr","type":"uint256"}],"name":"BaseAprSnapshotTaken","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"bool","name":"destinationMessageSend","type":"bool"}],"name":"DestinationMessageSendSet","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"uint256","name":"priorTimestamp","type":"uint256"},{"indexed":false,"internalType":"uint24","name":"discount","type":"uint24"},{"indexed":false,"internalType":"uint256","name":"currentTimestamp","type":"uint256"}],"name":"DiscountSnapshotTaken","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"uint8","name":"version","type":"uint8"}],"name":"Initialized","type":"event"},{"inputs":[],"name":"ALPHA","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"APR_FILTER_INIT_INTERVAL_IN_SEC","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"APR_SNAPSHOT_INTERVAL_IN_SEC","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"DISCOUNT_SNAPSHOT_INTERVAL_IN_SEC","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"accessController","outputs":[{"internalType":"contract IAccessController","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"baseApr","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"baseAprFilterInitialized","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"calculateEthPerToken","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"current","outputs":[{"components":[{"internalType":"uint256","name":"lastSnapshotTimestamp","type":"uint256"},{"internalType":"uint256","name":"baseApr","type":"uint256"},{"internalType":"int256","name":"discount","type":"int256"},{"internalType":"uint24[10]","name":"discountHistory","type":"uint24[10]"},{"internalType":"uint40","name":"discountTimestampByPercent","type":"uint40"}],"internalType":"struct ILSTStats.LSTStatsData","name":"","type":"tuple"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"destinationMessageSend","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"uint256","name":"","type":"uint256"}],"name":"discountHistory","outputs":[{"internalType":"uint24","name":"","type":"uint24"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"discountTimestampByPercent","outputs":[{"internalType":"uint40","name":"","type":"uint40"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"getAddressId","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"getAprId","outputs":[{"internalType":"bytes32","name":"","type":"bytes32"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"getSystemRegistry","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"bytes32[]","name":"","type":"bytes32[]"},{"internalType":"bytes","name":"initData","type":"bytes"}],"name":"initialize","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"lastBaseAprEthPerToken","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"lastBaseAprSnapshotTimestamp","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"lastDiscountSnapshotTimestamp","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"lstTokenAddress","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"setDestinationMessageSend","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"shouldSnapshot","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"snapshot","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"usePriceAsDiscount","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"pure","type":"function"}]"""


contract = eth_client.eth.contract(rETH_CALCULATOR, abi=abi)
discount_df = add_timestamp_to_df_with_block_column(fetch_events(contract.events.DiscountSnapshotTaken), ETH_CHAIN)
discount_df

2025-01-09 14:15:26.971 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-09 14:15:26.971 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-09 14:15:26.972 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-09 14:15:27.463 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-09 14:15:27.464 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


,priorTimestamp,discount,currentTimestamp,event,block,transaction_index,log_index,hash
timestamp,,,,,,,,
2024-08-31 12:51:47+00:00,0,0,1725108707,DiscountSnapshotTaken,20648670,86,212,0x6f13657889047704a3c03ebfafd2c0926bd7bab88536...
2024-09-01 12:52:23+00:00,1725108707,0,1725195143,DiscountSnapshotTaken,20655839,27,133,0x7f58be3105c5f2606d262487af6ec022ce9a73c14a7e...
2024-09-02 12:52:35+00:00,1725195143,0,1725281555,DiscountSnapshotTaken,20663002,16,114,0xb0d49e1d9364d88d7cdb851ba8722d86310b4a51056c...
2024-09-03 16:17:23+00:00,1725281555,0,1725380243,DiscountSnapshotTaken,20671190,100,282,0x3cd69ec00161ae2397b08aaf2477cb6d3c5913387324...
2024-09-04 18:13:11+00:00,1725380243,0,1725473591,DiscountSnapshotTaken,20678936,82,94,0xf398596f577573f62cd33e09c6afdcd51cc91f69536d...
...,...,...,...,...,...,...,...,...
2025-01-04 04:22:59+00:00,1735878167,30432,1735964579,DiscountSnapshotTaken,21548701,23,168,0xb0a2bbebd8c6f2657650c6c2db6515f9b6e7b2512ce8...
2025-01-05 04:23:11+00:00,1735964579,39720,1736050991,DiscountSnapshotTaken,21555868,28,188,0xb60db35fa0f38ac769c7b19c2a9e337b184291ff54d1...
2025-01-06 04:23:23+00:00,1736050991,35880,1736137403,DiscountSnapshotTaken,21563018,12,89,0x883f33e2dd0380e85a5337ac3675bbb4cf50230ddaa7...


In [11]:
discount_df["discount_percent"] = 100 * (discount_df["discount"] * (1e11)) / 1e18
px.line(discount_df["discount_percent"], title="rETH discount percent from events")

In [12]:
# StatCalculatorRegistered on StatsCalculatorRegistry




In [13]:
def nav_per_share_call(name: str, autopool_vault_address: str) -> Call:
    return Call(
        autopool_vault_address,
        ["convertToAssets(uint256)(uint256)", int(1e18)],
        [(name, safe_normalize_with_bool_success)],
    )

    
base_blocks = build_blocks_to_use(BASE_ETH.chain, end_block=BASE_ETH.chain.client.eth.block_number)

nav_per_share_calls = [nav_per_share_call('baseETH_nav_per_share', BASE_ETH.autopool_eth_addr)]
 
nav_per_share_df = get_raw_state_by_blocks(nav_per_share_calls, base_blocks, BASE_ETH.chain, include_block_number=True)

nav_per_share_df.to_csv('base_eth_nav_per_share.csv')
nav_per_share_df.tail(10)

2025-01-09 14:15:27.855 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-09 14:15:27.855 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-09 14:15:27.856 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-09 14:15:28.259 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-09 14:15:28.259 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


,baseETH_nav_per_share,block
timestamp,,
2025-01-06 11:52:33+00:00,1.011112,24687503
2025-01-06 17:52:33+00:00,1.011148,24698303
2025-01-06 23:52:33+00:00,1.011160,24709103
2025-01-07 05:52:33+00:00,1.010844,24719903
2025-01-07 11:52:33+00:00,1.010844,24730703
2025-01-07 17:52:33+00:00,1.010844,24741503
2025-01-07 23:52:33+00:00,1.010853,24752303
2025-01-08 05:52:33+00:00,1.010077,24763103
2025-01-08 11:52:33+00:00,1.010157,24773903


In [14]:
from mainnet_launch.pages.key_metrics.key_metrics import fetch_nav_per_share, add_new_nav_per_share_to_table

nav_per_share_df = fetch_nav_per_share(BASE_ETH)
nav_per_share_df

ImportError: cannot import name 'add_new_nav_per_share_to_table' from 'mainnet_launch.pages.key_metrics.key_metrics' (/home/parker/Documents/Tokemak/v2-rebalance-dashboard/mainnet_launch/pages/key_metrics/key_metrics.py)

In [57]:
BASE_ETH.chain.client.eth.block_number

24829615